Protein-Protein Interaction (PPI) Prediction
This notebook explores the PPI dataset, validates the data integrity, and trains a Random Forest model using various class-imbalance techniques (Class Weights, SMOTE, and SMOTE-Tomek)

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# Set visualization style
sns.set_theme(style="whitegrid")

Data Loading and Validation

First, we load the dataset and perform sanity checks to ensure probabilities and surface accessibility values are within their expected logical ranges.

In [24]:
ppi_data = pd.read_csv("data/ppi.csv")
display(ppi_data.head())

def check_dataframe(ppi_data):
    
    prob_cols = [col for col in ppi_data.columns if 'prob_' in col]
    surf_cols = [col for col in ppi_data.columns if 'surf_acc' in col]

    #check if there's any NaN value in the dataset
    ppi_data.isnull().values.any()
    
    #chekc probbilities value correctness
    prob_issues = 0
    for col in prob_cols:
        invalid_count = ((ppi_data[col] < 0) | (ppi_data[col] > 1)).sum()
        if invalid_count > 0:
            print(f"{col}: {invalid_count} values outside 0 and 1")
            prob_issues += 1
            
    if prob_issues == 0:
        print("probability columns have all values within range")

    #chekc psurface values correctness
    surf_issues = 0
    for col in surf_cols:
        invalid_count = (ppi_data[col] < 0).sum()
        if invalid_count > 0:
            print(f"{col}: {invalid_count} negative values")
            surf_issues += 1
            
    if surf_issues == 0:
        print("Surface columns have all values within range")

print(check_dataframe(ppi_data))


,Unnamed: 0,domain,aa_ProtPosition,uniprot_id,Rlength,sequence,normalized_length,normalized_abs_surf_acc,normalized_hydropathy_index,rel_surf_acc,...,9_wm_pssm_K,9_wm_pssm_M,9_wm_pssm_F,9_wm_pssm_P,9_wm_pssm_S,9_wm_pssm_T,9_wm_pssm_W,9_wm_pssm_Y,9_wm_pssm_V,p_interface
0,0,0,1,A0A024RAV5,188,M,0.238095,0.784319,0.711111,0.803,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
1,1,0,2,A0A024RAV5,188,T,0.238095,0.359207,0.422222,0.530,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
2,2,0,3,A0A024RAV5,188,E,0.238095,0.395387,0.111111,0.464,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
3,3,0,4,A0A024RAV5,188,Y,0.238095,0.401655,0.355556,0.385,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0
4,4,1,5,A0A024RAV5,188,K,0.238095,0.288103,0.066667,0.287,...,0.302858,0.436017,0.238759,0.103980,0.105653,0.161544,0.060391,0.140326,0.481904,0


probability columns have all values within range
Surface columns have all values within range
None


Preprocessing and Train/Test Split

To prevent data leakage, we split the data based on unique proteins (`uniprot_id`) rather than randomized rows. 33% of the proteins are for testing, 66% for training.

In [25]:
# class imbalance for total datset
counts = ppi_data['p_interface'].value_counts()
percentages = ppi_data['p_interface'].value_counts(normalize=True) * 100
print(counts)
print(percentages)

# Get list of unique proteins (228 total)
unique_proteins = ppi_data['uniprot_id'].unique()

# Split proteins (33% test, 67% train)
train_ids, test_ids = train_test_split(unique_proteins, test_size=0.33, random_state=22)
train_data = ppi_data[ppi_data['uniprot_id'].isin(train_ids)]
test_data = ppi_data[ppi_data['uniprot_id'].isin(test_ids)]

# Drop features not used for modeling
cols_to_drop = ['p_interface', 'uniprot_id', 'domain', 'sequence', 'aa_ProtPosition', 'Unnamed: 0']
  
X_train = train_data.drop(columns=cols_to_drop)
y_train = train_data['p_interface']
X_test = test_data.drop(columns=cols_to_drop)
y_test = test_data['p_interface']

# Check class imbalance
print("--- TARGET CLASS IMBALANCE ---")
print(f"Train Balance:\n{y_train.value_counts(normalize=True) * 100}")
print(f"\nTest Balance:\n{y_test.value_counts(normalize=True) * 100}")
print(f"\nShapes: X_train: {X_train.shape} | X_test: {X_test.shape}")


p_interface
0    57305
1     7845
Name: count, dtype: int64
p_interface
0    87.958557
1    12.041443
Name: proportion, dtype: float64
--- TARGET CLASS IMBALANCE ---
Train Balance:
p_interface
0    87.955821
1    12.044179
Name: proportion, dtype: float64

Test Balance:
p_interface
0    87.964179
1    12.035821
Name: proportion, dtype: float64

Shapes: X_train: (43822, 132) | X_test: (21328, 132)


In [26]:
#pre-processing

# get the list of all unique proteins (228 total)
unique_proteins = ppi_data['uniprot_id'].unique()

# takes 33% of the total proteins for test set, 67% in the training, n.22
train_ids, test_ids = train_test_split(unique_proteins, test_size=0.33, random_state=22)
train_data = ppi_data[ppi_data['uniprot_id'].isin(train_ids)]
test_data = ppi_data[ppi_data['uniprot_id'].isin(test_ids)]

# some fetaures have to be dropped
cols_to_drop = ['p_interface', 'uniprot_id', 'domain', 'sequence', 'aa_ProtPosition', 'Unnamed: 0']
  
X_train = train_data.drop(columns=cols_to_drop)
y_train = train_data['p_interface']

X_test = test_data.drop(columns=cols_to_drop)
y_test = test_data['p_interface']

# 1. Just call .value_counts() directly on the Series
train_counts = y_train.value_counts()
test_counts = y_test.value_counts()

# 2. Get the proportions (percentages) to easily check the imbalance
train_props = y_train.value_counts(normalize=True) * 100
test_props = y_test.value_counts(normalize=True) * 100

# 3. Print everything out clearly
print("--- RAW COUNTS ---")
print("Train 0s and 1s:\n", train_counts, "Train Class Balance:\n", train_props)
print("\nTest 0s and 1s:\n", test_counts, "\nTest Class Balance:\n", test_props)

print("\nShapes:", X_train.shape, X_test.shape)

--- RAW COUNTS ---
Train 0s and 1s:
 p_interface
0    38544
1     5278
Name: count, dtype: int64 Train Class Balance:
 p_interface
0    87.955821
1    12.044179
Name: proportion, dtype: float64

Test 0s and 1s:
 p_interface
0    18761
1     2567
Name: count, dtype: int64 
Test Class Balance:
 p_interface
0    87.964179
1    12.035821
Name: proportion, dtype: float64

Shapes: (43822, 132) (21328, 132)


Model Training and Handling Imbalance

Because the `p_interface` target is highly imbalanced, we will evaluate a Random Forest classifier using three different techniques:
1. Built-in Class Weights
2. SMOTE (Synthetic Minority Over-sampling Technique)
3. SMOTE-Tomek (Over-sampling combined with under-sampling)

In [27]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_balanced = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=22, n_jobs=-1)
rf_balanced.fit(X_train, y_train)
pred_balanced = rf_balanced.predict(X_test)

smote = SMOTE(random_state=23)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
rf_smote = RandomForestClassifier(n_estimators=100, random_state=23, n_jobs=-1)
rf_smote.fit(X_train_smote, y_train_smote)
pred_smote = rf_smote.predict(X_test)

smt = SMOTETomek(random_state=24)
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)
rf_smt = RandomForestClassifier(n_estimators=100, random_state=24, n_jobs=-1)
rf_smt.fit(X_train_smt, y_train_smt)
pred_smt = rf_smt.predict(X_test)

print("Class Weight Balanced:")
print(classification_report(y_test, pred_balanced))
print("SMOTE:")
print(classification_report(y_test, pred_smote))
print("SMOTE Tomek:")
print(classification_report(y_test, pred_smt))

Class Weight Balanced:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     18761
           1       0.49      0.01      0.03      2567

    accuracy                           0.88     21328
   macro avg       0.69      0.51      0.48     21328
weighted avg       0.83      0.88      0.83     21328

SMOTE:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     18761
           1       0.29      0.08      0.12      2567

    accuracy                           0.87     21328
   macro avg       0.59      0.53      0.52     21328
weighted avg       0.81      0.87      0.83     21328

SMOTE Tomek:
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     18761
           1       0.30      0.07      0.12      2567

    accuracy                           0.87     21328
   macro avg       0.59      0.53      0.52     21328
weighted avg       0.81      0.